# Dependencies

In [14]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import utils
import pickle

# Importing Data

In [2]:
stats_cnx = sql.connect('../../Data/Combined Clean Historical Data/Historical_Fantasy_Stats.db')
stats = pd.read_sql_query('SELECT * FROM Historical_Fantasy_Points ', stats_cnx)
stats_cnx.close()
stats['season'] = pd.to_datetime(stats['season'], format='%Y')
stats_prep = stats.copy()

# Manipulating Data

In [3]:
stats_prep['season'] = pd.to_datetime(stats_prep['season'], format='%Y')
stats_prep['nfanpts'] = stats_prep.groupby(['Name','player_id']).shift(-1)['fantast_pts']
stats_prep = stats_prep[stats_prep['nfanpts'].notna()]

# This is to take the players who record more then 40 fantasy points in a season.
stats_prep = stats_prep[stats_prep['nfanpts'] >= 40]
stats_prep

,Name,Position,player_id,season,fantast_pts,completions,attempts,passing_yards,passing_tds,interceptions,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,nfanpts
3,Donald Driver,WR,4541,2010-01-01,129.50,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,Tony Gonzalez,TE,6101,2010-01-01,171.60,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,Tony Gonzalez,TE,6101,2011-01-01,209.50,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,Tony Gonzalez,TE,6101,2012-01-01,234.00,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,Matt Hasselbeck,QB,7091,2010-01-01,148.04,266,444,3001,12,17,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,Josh Jacobs,RB,35700,2019-01-01,191.60,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,Miles Boykin,WR,35703,2019-01-01,50.80,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,Drew Lock,QB,35704,2019-01-01,68.00,100,156,1020,7,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,Daniel Jones,QB,35710,2019-01-01,210.98,284,459,3027,24,12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [5]:
stats_prep.rename(columns = {'nfanpts':'target'}, inplace = True)

In [6]:
unwanted_col = ['Name', 'Position', 'player_id','fantast_pts','rushing_epa', 
           'receiving_2pt_conversions', 'special_teams_tds', 'rushing_2pt_conversions',
           'rushing_fumbles_lost']

stats_prep.drop(unwanted_col, axis=1, inplace=True)
predictors = list(stats_prep)
unwanted_predictor = {'season', 'target'}
predictors = [e for e in predictors if e not in unwanted_predictor]

In [7]:
stats_prep

,season,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
3,2010-01-01,0,0,0,0,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,2010-01-01,0,0,0,0,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,2011-01-01,0,0,0,0,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,2012-01-01,0,0,0,0,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,2010-01-01,266,444,3001,12,17,29,175,6,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,2019-01-01,0,0,0,0,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,2019-01-01,0,0,0,0,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,2019-01-01,100,156,1020,7,3,5,26,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,2019-01-01,284,459,3027,24,12,38,295,12,8,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [8]:
stats_prep = stats_prep.set_index(['season'])
stats_prep = stats_prep.sort_index()

In [9]:
stats_prep.apply(pd.isnull).sum()

completions                    0
attempts                       0
passing_yards                  0
passing_tds                    0
interceptions                  0
sacks                          0
sack_yards                     0
sack_fumbles                   0
sack_fumbles_lost              0
passing_air_yards              0
passing_yards_after_catch      0
passing_first_downs            0
passing_epa                    0
passing_2pt_conversions        0
carries                        0
rushing_yards                  0
rushing_tds                    0
rushing_fumbles                0
rushing_first_downs            0
receptions                     0
targets                        0
receiving_yards                0
receiving_tds                  0
receiving_fumbles              0
receiving_fumbles_lost         0
receiving_air_yards            0
receiving_yards_after_catch    0
receiving_first_downs          0
receiving_epa                  0
games                          0
tgt_sh    

In [10]:
stats_prep['rtd_sh'] = stats_prep['rtd_sh'].fillna(stats_prep['rtd_sh'].mean())
stats_prep['dom'] = stats_prep['dom'].fillna(stats_prep['dom'].mean())
stats_prep['w8dom'] = stats_prep['w8dom'].fillna(stats_prep['w8dom'].mean())

In [11]:
X = stats_prep.drop("target", axis=1)
Y = stats_prep["target"]

# Gridded KNN Initialization

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [15]:
parameters = {"n_neighbors": range(1, 41)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 41)})

In [16]:
gridsearch.best_params_

{'n_neighbors': 34}

In [46]:
train_preds_grid = gridsearch.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds_grid)
train_rmse = np.sqrt(train_mse)
test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = np.sqrt(test_mse)

In [18]:
print(f'Training RMSE: {train_rmse}\n Testing RMSE: {test_rmse}')

Training RMSE: 63.235102360916066
 Testing RMSE: 64.30557401022247


In [23]:
combined = pd.concat([y_test, pd.Series(test_preds_grid, index=y_test.index)], axis=1)
combined.columns = ["Test Real Data", "Predictions for Test Data"]

In [24]:
combined.describe()

,Test Real Data,Predictions for Test Data
count,732.000000,732.000000
mean,142.102213,142.025865
std,79.825124,45.746213
min,40.000000,73.295294
25%,77.650000,103.945588
50%,124.000000,133.866471
75%,193.500000,172.257206
max,407.800000,277.962353


In [25]:
combined.agg(['mean', 'median', 'std', 'var', 'skew', 'kurt'])

,Test Real Data,Predictions for Test Data
mean,142.102213,142.025865
median,124.000000,133.866471
std,79.825124,45.746213
var,6372.050431,2092.715977
skew,0.795438,0.747279
kurt,-0.063140,-0.108465


In [27]:
filename = "knn_grid_model.pkl"
pickle.dump(gridsearch, open(filename, 'wb'))

# Gridded KNN 2022 Predictions

In [28]:
stats_cnx = sql.connect('../../Data/Yearly Data as Database/Stats_with_names_2021.db')
stats_2021 = pd.read_sql_query('SELECT * FROM Stats_with_names_2021 ', stats_cnx)
stats_cnx.close()

In [29]:
stats_2021['wopr'] = stats_2021['wopr_x'] + stats_2021['wopr_y']

In [30]:
stats_2021['rtd_sh'] = stats_2021['rtd_sh'].fillna(stats_2021['rtd_sh'].mean())
stats_2021['dom'] = stats_2021['dom'].fillna(stats_2021['dom'].mean())
stats_2021['w8dom'] = stats_2021['w8dom'].fillna(stats_2021['w8dom'].mean())

In [31]:
grid_pred_2022 = gridsearch.predict(stats_2021[predictors])

In [32]:
stats_2021['rob_fpts_2022'] = grid_pred_2022

In [33]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,274.04,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,76.02,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,204.69,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,235.29,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,75.13,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,79.81,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,92.39,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,85.45,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,130.17,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [35]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
87,Derek Carr,QB,275.92
538,Justin Herbert,QB,275.39
14,Matthew Stafford,QB,274.98
0,Tom Brady,QB,274.04
255,Patrick Mahomes,QB,273.49
...,...,...,...
383,Davion Davis,WR,74.58
416,Chris Myarick,TE,74.58
491,Easop Winston,WR,74.27
480,Gabe Nabers,RB,72.81


In [36]:
top50 = predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    21
WR    20
RB     6
TE     3
Name: Position, dtype: int64

In [37]:
cnx = sql.connect("Predicted_fpts_2022_KNN_Grid_Search.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out
predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_Grid_Search', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   

# Weighted Grid KNN

In [40]:
parameters = {"n_neighbors": range(1, 41), "weights": ["uniform", "distance"]}

In [41]:
weighted_gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
weighted_gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 41),
                         'weights': ['uniform', 'distance']})

In [42]:
weighted_gridsearch.best_params_

{'n_neighbors': 39, 'weights': 'distance'}

In [50]:
test_preds_grid_weighted = weighted_gridsearch.predict(X_test)
weighted_test_mse = mean_squared_error(y_test, test_preds_grid_weighted)
weighted_test_rmse = np.sqrt(weighted_test_mse)
print(f'Non-weighted Test RMSE: {test_rmse}\n Weighted Test RMSE: {weighted_test_rmse}')

Non-weighted Test RMSE: 64.30557401022247
 Weighted Test RMSE: 64.265017317235


In [51]:
weighted_combined = pd.concat([y_test, pd.Series(test_preds_grid_weighted, index=y_test.index)], axis=1)
weighted_combined.columns = ["Test Real Data", "Predictions for Test Data"]

In [52]:
weighted_combined.describe()

,Test Real Data,Predictions for Test Data
count,732.000000,732.000000
mean,142.102213,142.412011
std,79.825124,45.866938
min,40.000000,72.057638
25%,77.650000,103.270157
50%,124.000000,134.213779
75%,193.500000,171.636240
max,407.800000,279.525166


In [53]:
weighted_combined.agg(['mean', 'median', 'std', 'var', 'skew', 'kurt'])

,Test Real Data,Predictions for Test Data
mean,142.102213,142.412011
median,124.000000,134.213779
std,79.825124,45.866938
var,6372.050431,2103.776007
skew,0.795438,0.757910
kurt,-0.063140,-0.120273


In [54]:
filename = "knn_weighted_grid_model.pkl"
pickle.dump(weighted_gridsearch, open(filename, 'wb'))

## Weighted 2022 Predictions

In [56]:
weighted_grid_pred_2022 = weighted_gridsearch.predict(stats_2021[predictors])

In [57]:
stats_2021['rob_fpts_2022'] = weighted_grid_pred_2022

In [58]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,273.86,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,88.03,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,195.82,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,231.20,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,77.89,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,77.64,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,93.66,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,87.08,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,148.49,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
weighted_predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)
weighted_predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
538,Justin Herbert,QB,277.58
255,Patrick Mahomes,QB,275.03
0,Tom Brady,QB,273.86
87,Derek Carr,QB,273.44
14,Matthew Stafford,QB,272.72
...,...,...,...
530,Braden Mann,PN,72.90
311,Ito Smith,RB,72.69
595,Adam Prentice,RB,71.80
173,Derek Watt,RB,71.57


In [63]:
top50 = weighted_predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    23
WR    18
RB     7
TE     2
Name: Position, dtype: int64

In [64]:
cnx = sql.connect("Predicted_fpts_2022_KNN_Weighted_Grid_Search.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out
weighted_predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_Weighted_Grid_Search', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   